In [1]:
from gnews import GNews
import pandas as pd
import numpy as np
from newspaper import Article
import re
import time

## Using GNews to collect news article headlines

https://github.com/ranahaani/GNews   

In [76]:
pd.read_excel('news and keywords.xlsx')['Keywords (v2)'].dropna().to_list()

['palestine',
 'student',
 'university',
 'college',
 'encampment',
 'divest',
 'anti',
 'pro',
 'israel',
 'demonstrator',
 'activist',
 'peaceful',
 'against',
 'unite',
 'solidarity',
 'sjp',
 'divestment',
 'campus',
 'arrested ',
 'detained',
 'sit in',
 'gaza']

In [28]:
?GNews

In [137]:
article = Article('https://apnews.com/article/israel-palestinian-campus-protests-timeline-f7cd3abe635f8afa4532b7bed9212b56')

In [143]:
article.keywords

[]

In [2]:
google_news = GNews(language='en', 
                    country='US', 
                    #period='7d', 
                    start_date=(2024, 3, 1), 
                    end_date=(2024, 3, 30), 
                    #max_results=10, 
                    #exclude_websites=['yahoo.com', 'cnn.com'],
                    #proxy=proxy
                   )

results = google_news.get_news('encampment')

In [60]:
# check for duplicates in O(1) time 
collected_articles = dict()
collected_articles[results[0]['url']] = True
results[1]['url'] in collected_articles


In [7]:
start = time.time()
# generate dates in 2 week intervals from March 01, 2024 to May 23rd, 2024
dates = pd.date_range(start='3/1/2024', end='5/23/2024', freq='2W')
# turn into tuples
dates = [(date.year, date.month, date.day) for date in dates]


# for each date interval, search and collect data
for i in range(len(dates) - 1): # stop before the last date
    
#     print('this is date: ', dates[i])
#     print('end: ', dates[i + 1])
    
    # create gnews object 
    news = GNews(language='en', 
                    country='US', 
                    start_date= dates[i], 
                    end_date=dates[i + 1], 
                    max_results=100, 
                    #exclude_websites=['yahoo.com', 'cnn.com'],
                    #proxy=proxy
                   )
    # get news 
    results = news.get_news('student protest')
    
    # if this is the first iteration
    if i == 0: 
        df = pd.DataFrame(results)
    
    
    # else add to existing df 
    else: 
        new_df = pd.DataFrame(results)
        df = pd.concat([df, new_df])
    
    # increment i 
    i += 1

end = time.time()
print('time elapsed: ', end - start, ' seconds')
df.head()

time elapsed:  61.71826887130737  seconds


,title,description,published date,url,publisher
0,How a student protest plan helped get a playgr...,How a student protest plan helped get a playgr...,"Mon, 04 Mar 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMiaGh0d...,"{'href': 'https://ny1.com', 'title': 'Spectrum..."
1,Pro-Palestine student organizers face crackdow...,Pro-Palestine student organizers face crackdow...,"Tue, 05 Mar 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMiW2h0d...,"{'href': 'https://prismreports.org', 'title': ..."
2,A parent's perspective on Family Weekend activ...,A parent's perspective on Family Weekend activ...,"Fri, 15 Mar 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMiTWh0d...,"{'href': 'https://stanforddaily.com', 'title':..."
3,"Students, staff and faculty protest DEI decisi...","Students, staff and faculty protest DEI decisi...","Fri, 08 Mar 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMifmh0d...,"{'href': 'https://www.alligator.org', 'title':..."
4,Politics of Silence: Student Protest Stops Spe...,Politics of Silence: Student Protest Stops Spe...,"Fri, 15 Mar 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMibmh0d...,"{'href': 'https://mastmedia.plu.edu', 'title':..."


In [45]:
df

,title,description,published date,url,publisher
0,How a student protest plan helped get a playgr...,How a student protest plan helped get a playgr...,"Mon, 04 Mar 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMiaGh0d...,"{'href': 'https://ny1.com', 'title': 'Spectrum..."
1,Pro-Palestine student organizers face crackdow...,Pro-Palestine student organizers face crackdow...,"Tue, 05 Mar 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMiW2h0d...,"{'href': 'https://prismreports.org', 'title': ..."
2,A parent's perspective on Family Weekend activ...,A parent's perspective on Family Weekend activ...,"Fri, 15 Mar 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMiTWh0d...,"{'href': 'https://stanforddaily.com', 'title':..."
3,"Students, staff and faculty protest DEI decisi...","Students, staff and faculty protest DEI decisi...","Fri, 08 Mar 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMifmh0d...,"{'href': 'https://www.alligator.org', 'title':..."
4,Politics of Silence: Student Protest Stops Spe...,Politics of Silence: Student Protest Stops Spe...,"Fri, 15 Mar 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMibmh0d...,"{'href': 'https://mastmedia.plu.edu', 'title':..."
...,...,...,...,...,...
95,"In the 1980s, he led student protests. Now, he...","In the 1980s, he led student protests. Now, he...","Sun, 05 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMiUWh0d...,"{'href': 'https://www.npr.org', 'title': 'NPR'}"
96,Statement to Board of Governors on Student Pro...,Statement to Board of Governors on Student Pro...,"Mon, 06 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMiVWh0d...,"{'href': 'https://www.rutgers.edu', 'title': '..."
97,"Student protesters demand UVM, Middlebury Coll...","Student protesters demand UVM, Middlebury Coll...","Mon, 29 Apr 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMiZmh0d...,"{'href': 'https://www.wcax.com', 'title': 'WCAX'}"
98,"Once champions of free speech, colleges crack ...","Once champions of free speech, colleges crack ...","Tue, 07 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMiaWh0d...,"{'href': 'https://www.nbcnews.com', 'title': '..."


In [13]:
test = [{'title': 'College Dorm Decorations Become a Front in the Campus Free Speech Wars - The New York Times',
  'description': 'College Dorm Decorations Become a Front in the Campus Free Speech Wars  The New York Times',
  'published date': 'Fri, 01 Mar 2024 08:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMiVGh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDMvMDEvbnlyZWdpb24vY2FtcHVzLWZyZWUtc3BlZWNoLWJhcm5hcmQtY29sdW1iaWEuaHRtbNIBAA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.nytimes.com',
   'title': 'The New York Times'},
    'ot': True}, 
 {'title': 'UPenn Trustees Meeting Cut Short by Student Protest Over Israel-Hamas War - The New York Times',
  'description': 'UPenn Trustees Meeting Cut Short by Student Protest Over Israel-Hamas War  The New York Times',
  'published date': 'Fri, 01 Mar 2024 08:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMiRWh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDMvMDEvdXMvdXBlbm4tc3R1ZGVudC1wcm90ZXN0LWdhemEuaHRtbNIBAA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.nytimes.com',
   'title': 'The New York Times'},
     'ot': True},
 {'title': 'Jewish Students Describe Facing Antisemitism on Campus to Members of Congress - The New York Times',
  'description': 'Jewish Students Describe Facing Antisemitism on Campus to Members of Congress  The New York Times',
  'published date': 'Fri, 01 Mar 2024 08:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMiTmh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDMvMDEvdXMvYW50aXNlbWl0aXNtLWNhbXB1cy1qZXdpc2gtc3R1ZGVudHMuaHRtbNIBAA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.nytimes.com',
   'title': 'The New York Times'}, 
     'ot': False},
 {'title': 'South Korean doctors hold mass protest against government’s medical school admissions plan - PBS NewsHour',
  'description': 'South Korean doctors hold mass protest against government’s medical school admissions plan  PBS NewsHour',
  'published date': 'Sun, 03 Mar 2024 08:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMifGh0dHBzOi8vd3d3LnBicy5vcmcvbmV3c2hvdXIvd29ybGQvc291dGgta29yZWFuLWRvY3RvcnMtaG9sZC1tYXNzLXByb3Rlc3QtYWdhaW5zdC1nb3Zlcm5tZW50cy1tZWRpY2FsLXNjaG9vbC1hZG1pc3Npb25zLXBsYW7SAYABaHR0cHM6Ly93d3cucGJzLm9yZy9uZXdzaG91ci9hbXAvd29ybGQvc291dGgta29yZWFuLWRvY3RvcnMtaG9sZC1tYXNzLXByb3Rlc3QtYWdhaW5zdC1nb3Zlcm5tZW50cy1tZWRpY2FsLXNjaG9vbC1hZG1pc3Npb25zLXBsYW4?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.pbs.org', 'title': 'PBS NewsHour'},
    'ot': False},
 {'title': "'What if it was Duke?': Faculty, students and staff protest deaths of educators and students in Gaza - Duke Chronicle",
  'description': "'What if it was Duke?': Faculty, students and staff protest deaths of educators and students in Gaza  Duke Chronicle",
  'published date': 'Fri, 01 Mar 2024 08:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMisAFodHRwczovL3d3dy5kdWtlY2hyb25pY2xlLmNvbS9hcnRpY2xlLzIwMjQvMDMvZHVrZS11bml2ZXJzaXR5LWZhY3VsdHktc3R1ZGVudHMtc3RhZmYtcHJvdGVzdC1kZWF0aHMtZWR1Y2F0b3JzLXN0dWRlbnRzLWdhemEtaXNyYWVsLWhhbWFzLXdhci1wYWxlc3RpbmUtc3RvcC1zY2hvbGFzdGljaWRlLWNoYXBlbNIBAA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.dukechronicle.com',
   'title': 'Duke Chronicle'}, 
     'ot': True},
    {'title': 'Associated Students of the University of Alaska Fairbanks protest “impending” tuition increase - Webcenter Fairbanks',
  'description': 'Associated Students of the University of Alaska Fairbanks protest “impending” tuition increase  Webcenter Fairbanks',
  'published date': 'Tue, 05 Mar 2024 08:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMigQFodHRwczovL3d3dy53ZWJjZW50ZXJmYWlyYmFua3MuY29tLzIwMjQvMDMvMDUvYXNzb2NpYXRlZC1zdHVkZW50cy11bml2ZXJzaXR5LWFsYXNrYS1mYWlyYmFua3MtcHJvdGVzdC1pbXBlbmRpbmctdHVpdGlvbi1pbmNyZWFzZS_SAZABaHR0cHM6Ly93d3cud2ViY2VudGVyZmFpcmJhbmtzLmNvbS8yMDI0LzAzLzA1L2Fzc29jaWF0ZWQtc3R1ZGVudHMtdW5pdmVyc2l0eS1hbGFza2EtZmFpcmJhbmtzLXByb3Rlc3QtaW1wZW5kaW5nLXR1aXRpb24taW5jcmVhc2UvP291dHB1dFR5cGU9YW1w?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.webcenterfairbanks.com',
   'title': 'Webcenter Fairbanks'}, 
    'ot': False},
     {'title': 'More than 120 protesters blinded by Iranian agents, probe confirms - UC Berkeley',
  'description': 'More than 120 protesters blinded by Iranian agents, probe confirms  UC Berkeley',
  'published date': 'Thu, 21 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMiZ2h0dHBzOi8vbmV3cy5iZXJrZWxleS5lZHUvMjAyNC8wMy8yMS9tb3JlLXRoYW4tMTIwLXByb3Rlc3RlcnMtYmxpbmRlZC1ieS1pcmFuaWFuLWFnZW50cy1wcm9iZS1jb25maXJtcy_SAQA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://news.berkeley.edu', 'title': 'UC Berkeley'},
     'ot': False},
        {'title': 'Protest of Chinese student recruitment ban draws crowd outside Board of Governors meeting at UF - The Independent Florida Alligator',
  'description': 'Protest of Chinese student recruitment ban draws crowd outside Board of Governors meeting at UF  The Independent Florida Alligator',
  'published date': 'Tue, 26 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMiOWh0dHBzOi8vd3d3LmFsbGlnYXRvci5vcmcvYXJ0aWNsZS8yMDI0LzAzL2ZhY3VsdHktcHJvdGVzdNIBAA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.alligator.org',
   'title': 'The Independent Florida Alligator'},
        'ot': False},
        {'title': 'Vanderbilt students protest over Israel-divestment: What to know - Tennessean',
  'description': 'Vanderbilt students protest over Israel-divestment: What to know  Tennessean',
  'published date': 'Thu, 28 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMikAFodHRwczovL3d3dy50ZW5uZXNzZWFuLmNvbS9zdG9yeS9uZXdzL2VkdWNhdGlvbi8yMDI0LzAzLzI4L3ZhbmRlcmJpbHQtc3R1ZGVudC1wcm90ZXN0LW92ZXItaXNyYWVsLWRpdmVzdG1lbnQtbmFzaHZpbGxlLXdoYXQtdG8ta25vdy83MzEzMDk3MzAwNy_SAQA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.tennessean.com', 'title': 'Tennessean'},
        'ot': True},
        {'title': 'How a student protest plan helped get a playground reopened - Spectrum News NY1',
  'description': 'How a student protest plan helped get a playground reopened  Spectrum News NY1',
  'published date': 'Mon, 04 Mar 2024 08:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMiaGh0dHBzOi8vbnkxLmNvbS9ueWMvbWFuaGF0dGFuL25ld3MvMjAyNC8wMy8wNC90aGUtY2hpbGRyZW5zLXdvcmtzaG9wLXNjaG9vbC1wbGF5Z3JvdW5kLXJlb3BlbmluZy1wcm90ZXN00gEA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://ny1.com', 'title': 'Spectrum News NY1'},
        'ot': False},
        {'title': 'Get a look at St. Norbert College student protest over faculty layoffs. - Green Bay Press Gazette',
  'description': 'Get a look at St. Norbert College student protest over faculty layoffs.  Green Bay Press Gazette',
  'published date': 'Wed, 27 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMioQFodHRwczovL3d3dy5ncmVlbmJheXByZXNzZ2F6ZXR0ZS5jb20vcGljdHVyZS1nYWxsZXJ5L25ld3MvZWR1Y2F0aW9uLzIwMjQvMDMvMjcvc2VlLXBob3Rvcy1vZi1zdC1ub3JiZXJ0LWNvbGxlZ2Utc3R1ZGVudC1wcm90ZXN0LW92ZXItZmFjdWx0eS1sYXlvZmZzLzczMTIxMTgxMDA3L9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.greenbaypressgazette.com',
   'title': 'Green Bay Press Gazette'},
        'ot': False},
        {'title': "'Please Respond Personally' Exhibit Remembers 1974 Black Student Protests at UConn - UConn Today - University of Connecticut",
  'description': "'Please Respond Personally' Exhibit Remembers 1974 Black Student Protests at UConn - UConn Today  University of Connecticut",
  'published date': 'Wed, 27 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMicmh0dHBzOi8vdG9kYXkudWNvbm4uZWR1LzIwMjQvMDMvcGxlYXNlLXJlc3BvbmQtcGVyc29uYWxseS1leGhpYml0cy1yZW1lbWJlcnMtMTk3NC1ibGFjay1zdHVkZW50LXByb3Rlc3RzLWF0LXVjb25uL9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://today.uconn.edu',
   'title': 'University of Connecticut'},
        'ot': False},
        {'title': 'BREAKING: Students forced to exit Kirkland Hall, three arrested and 16 suspended - The Vanderbilt Hustler',
  'description': 'BREAKING: Students forced to exit Kirkland Hall, three arrested and 16 suspended  The Vanderbilt Hustler',
  'published date': 'Wed, 27 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMicWh0dHBzOi8vdmFuZGVyYmlsdGh1c3RsZXIuY29tLzIwMjQvMDMvMjcvYnJlYWtpbmctc3R1ZGVudHMtZm9yY2libHktbGVhdmUtc2l0LWluLXRocmVlLWFycmVzdGVkLWFuZC0xNi1zdXNwZW5kZWQv0gEA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://vanderbilthustler.com',
   'title': 'The Vanderbilt Hustler'},
        'ot': True},
        {'title': 'Student protest calls for compensation for art students - The Appalachian Online',
  'description': 'Student protest calls for compensation for art students  The Appalachian Online',
  'published date': 'Sat, 23 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMiWWh0dHBzOi8vdGhlYXBwYWxhY2hpYW5vbmxpbmUuY29tL3N0dWRlbnQtcHJvdGVzdC1jYWxscy1mb3ItY29tcGVuc2F0aW9uLWZvci1hcnQtc3R1ZGVudHMv0gEA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://theappalachianonline.com',
   'title': 'The Appalachian Online'},
        'ot': False},
        {'title': 'Pro-Palestinian student groups sue Columbia over suspension - Prism',
  'description': 'Pro-Palestinian student groups sue Columbia over suspension  Prism',
  'published date': 'Wed, 20 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMiVWh0dHBzOi8vcHJpc21yZXBvcnRzLm9yZy8yMDI0LzAzLzIwL3Byby1wYWxlc3RpbmUtc3R1ZGVudC1ncm91cHMtY29sdW1iaWEtc3VzcGVuc2lvbi_SAQA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://prismreports.org', 'title': 'Prism'},
        'ot': True},
        {'title': 'Dozens of students walk out of Pocahontas County HS in protest of staff, class cuts - WDTV',
  'description': 'Dozens of students walk out of Pocahontas County HS in protest of staff, class cuts  WDTV',
  'published date': 'Mon, 25 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMiZ2h0dHBzOi8vd3d3LndkdHYuY29tLzIwMjQvMDMvMjUvZG96ZW5zLXN0dWRlbnRzLXdhbGstb3V0LXBvY2Fob250YXMtY291bnR5LWhzLXByb3Rlc3Qtc3RhZmYtY2xhc3MtY3V0cy_SAXZodHRwczovL3d3dy53ZHR2LmNvbS8yMDI0LzAzLzI1L2RvemVucy1zdHVkZW50cy13YWxrLW91dC1wb2NhaG9udGFzLWNvdW50eS1ocy1wcm90ZXN0LXN0YWZmLWNsYXNzLWN1dHMvP291dHB1dFR5cGU9YW1w?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.wdtv.com', 'title': 'WDTV'},
        'ot': False},
 {'title': 'UW students protest against Muslim hate after racist letter sent to student group - FOX 13 Seattle',
  'description': 'UW students protest against Muslim hate after racist letter sent to student group  FOX 13 Seattle',
  'published date': 'Thu, 28 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMidmh0dHBzOi8vd3d3LmZveDEzc2VhdHRsZS5jb20vbmV3cy9zdHVkZW50cy1wcm90ZXN0LWF0LXV3LWFnYWluc3QtbXVzbGltLWhhdGUtYWZ0ZXItcmFjaXN0LWxldHRlci1zZW50LXRvLXN0dWRlbnQtZ3JvdXDSAXpodHRwczovL3d3dy5mb3gxM3NlYXR0bGUuY29tL25ld3Mvc3R1ZGVudHMtcHJvdGVzdC1hdC11dy1hZ2FpbnN0LW11c2xpbS1oYXRlLWFmdGVyLXJhY2lzdC1sZXR0ZXItc2VudC10by1zdHVkZW50LWdyb3VwLmFtcA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.fox13seattle.com',
   'title': 'FOX 13 Seattle'},
     'ot': True},
 {'title': 'New Orleans community member arrested at protest for Tulane student - Maroon',
  'description': 'New Orleans community member arrested at protest for Tulane student  Maroon',
  'published date': 'Sat, 16 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMidWh0dHBzOi8vbG95b2xhbWFyb29uLmNvbS8xMDA0MTk1MS93b3JsZHZpZXcvY2l0eS9uZXctb3JsZWFucy1jb21tdW5pdHktbWVtYmVyLWFycmVzdGVkLWF0LXByb3Rlc3QtZm9yLXR1bGFuZS1zdHVkZW50L9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://loyolamaroon.com', 'title': 'Maroon'},
     'ot': False},
        {'title': "'You are funding genocide': TAHRIR members protest during 101st Annual Honors Convocation - The Michigan Daily",
  'description': "'You are funding genocide': TAHRIR members protest during 101st Annual Honors Convocation  The Michigan Daily",
  'published date': 'Mon, 25 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMiiQFodHRwczovL3d3dy5taWNoaWdhbmRhaWx5LmNvbS9uZXdzL2FkbWluaXN0cmF0aW9uL3lvdS1hcmUtZnVuZGluZy1nZW5vY2lkZS10YWhyaXItbWVtYmVycy1wcm90ZXN0LWR1cmluZy0xMDFzdC1hbm51YWwtaG9ub3JzLWNvbnZvY2F0aW9uL9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.michigandaily.com',
   'title': 'The Michigan Daily'},
        'ot': True},
 {'title': "Students are protesting Cornell's response to the war in Gaza. Now they say they're being disciplined - WSKG.org",
  'description': "Students are protesting Cornell's response to the war in Gaza. Now they say they're being disciplined  WSKG.org",
  'published date': 'Mon, 18 Mar 2024 07:00:00 GMT',
  'url': 'https://news.google.com/rss/articles/CBMihwFodHRwczovL3d3dy53c2tnLm9yZy9uZXdzLzIwMjQtMDMtMTgvc3R1ZGVudHMtYXJlLXByb3Rlc3RpbmctY29ybmVsbHMtcmVzcG9uc2UtdG8tdGhlLXdhci1pbi1nYXphLW5vdy10aGV5LXNheS10aGV5cmUtYmVpbmctZGlzY2lwbGluZWTSAQA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'publisher': {'href': 'https://www.wskg.org', 'title': 'WSKG.org'},
     'ot': True}
]

test_df = pd.DataFrame(test)

### Data Cleaning

Our Approach: how do we determine if an article is about student protest?

Part 1: 

If it contains two components: 

1.) has student related keywords

2.) has protest or 

israel related keyword

Part 2: 

For headlines with low score, get the full articles using the newspaper package to verify its topic
If the score is still low, we drop it 

student-related words: 
- student*
- college*
- universit*
- school*
- campus*
- faculty

protest-related words:
- activis*
- protest*
- encampment
- demonstrat*
- clash
- divest*
- \*war\*

context-related words: 
- pro-palestin*
- Israel-Hamas
- Gaza
- pro-israel


Create a confidence score  
Subscore: $\frac{\text{# of captured words in the category}}{\text{total # of words in the category}}$

$\frac{\text{student score} \; \times \;  \text{protest score} \; \times \; \text{context score}} {\text{total words in headline}}$  ?


--------------------------------------------------------------------------------------------

schools:
- columbia
- harvard
- brown
- northwestern
- UPenn
- UCLA
- UCSD
- Berkeley 

In [145]:
results[0]['title']

'Opinion: My experience at the Columbia University encampment - The Connecticut Mirror'

In [199]:
keywords: {
    'student.*': 'student',
    'college.*': 'student', 
    'universit.*': 'student',
    'school.*': 'student', 
    'campus.*': 'student', 
    'faculty': 'student', 
    'activis.*': 'protest', 
    'protest.*': 'protest', 
    'encampment.*': 'protest', 
    'demostrat*': 'protest', 
    'clash*': 'protest', 
    'divest*': 'protest', 
    '.*war.*': 'protest', 
    'palestin.*': 'context', 
    'israel': 'context', 
    'gaza': 'context'
}

In [200]:
'students' in keywords

False

In [19]:
keywords.keys()

dict_keys(['student', 'protest', 'context'])

In [75]:
pattern = re.compile(r'student*')
string = "students"

# Use the fullmatch method to ensure the entire string matches the pattern
match = pattern.fullmatch(string)


for key in keywords['context'].keys(): 
    pattern = re.compile(fr'{key}')
    string = 'palestine'
    match = pattern.fullmatch(string)
    if match: 
        print(key)

palestin.*


In [70]:
pattern = re.compile(r'student*')
re.fullmatch('demostrat.*', 'demonstrators')

In [23]:
keywords['student'].keys()

dict_keys(['student*', 'college*', 'universit*', 'school*', 'campus*', 'faculty'])

In [71]:
keywords = {
    'student': {
        'student.*': None,
        'college.*': None, 
        'universit.*': None,
        'school.*': None, 
        'campus.*': None, 
        'faculty': None
    }, 
    'protest': {
        'activis.*': None, 
        'protest.*': None, 
        'encamp.*': None, 
        'demonstrat.*': None, 
        'clash.*': None, 
        'divest.*': None, 
        'war.*': None
    }, 
    'context': {
        'palestin.*': None, 
        'israel.*': None, 
        'gaza': None,
    }
}

# def get_score(text):
#     # remove everything but words and hyphenated words from text, then turn into list of words
#     text = text.strip().lower()
#     text = re.sub(r'\W+', ' ', text).split(' ')
    
#     # calculate subscore for each category
#     student_score = 0
#     protest_score = 0
#     context_score = 0
        
#     def get_score(word): 
#         for cat in keywords.keys():
            
        
    
#     def subscore(text): 
#         student_score = 0
#         protest_score = 0
#         context_score = 0
        
#         for word in text: 
#             if word in keywords['student']
        
#         [for word in text if word ]
        
        
#         return student_score, protest_score, context_score
            
    
#     return text
    
# get_score('this is a nother sentenec aakk         alhflkdj ')

- foxnews.com
- cnn.com
- nytimes.com